<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/Assessment_Gemma_2_2B_T4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assessment on Gemma 2 2B

https://huggingface.co/google/gemma-2-2b-it/

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from IPython.display import Markdown

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in ca

In [4]:
!nvidia-smi

Fri Aug  9 06:25:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import transformers
transformers.__version__


'4.42.4'

In [6]:
model_id = "google/gemma-2-2b-it"

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

***note:*** execute in a termial 'watch -n 0.5 nvidia-smi' to see the GPU usage and when the model is loaded onto it

In [8]:
%%time

from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # quantization_config=quantization_config, # quantized 8-Bit for T4
    torch_dtype=torch.bfloat16,  # 16-Bit full resolution
    device_map="auto",
    trust_remote_code=True
)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

CPU times: user 7.3 s, sys: 7.23 s, total: 14.5 s
Wall time: 28.4 s


In [9]:
!nvidia-smi

Fri Aug  9 06:26:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0              27W /  70W |   5287MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
positive = [
  "With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.",
  "The socio-medical prerequisites for the prescribed aid supply have been met.",
  "Everyday relevant usage benefits have been determined.",
  "Socio-medical indication for the aid is confirmed.",
  "Contraindications have been excluded; there are no contraindications for the use of the requested aid."
]

In [11]:
negative = [
  "No specific findings can be derived from the diagnosis currently named as the basis for the regulation.",
  "According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.",
  "A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.",
  "From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.",
  "A medical justification for why a product not listed in the directory of aids should be used in the present case has not been transmitted."
]

In [12]:
assessment = negative[0]
# assessment = positive[0]

In [18]:
%%time

messages = [
    {"role": "user", "content": f'''
You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal.
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

# Assessment
{assessment}

'''},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=False
)
response = outputs[0][input_ids.shape[-1]:]
Markdown(tokenizer.decode(response, skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


CPU times: user 3.37 s, sys: 289 ms, total: 3.66 s
Wall time: 4.56 s


Yes. 

The assessment explicitly states that the need for compensation to ensure basic needs is "conceivable" due to the diagnosis. This indicates a positive recommendation for compensation. 


In [24]:
def eval_assessment(assessment):
  messages = [
    {"role": "user", "content": f'''
You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal.
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

# Assessment
{assessment}

'''},
]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=512,
      eos_token_id=terminators,
      pad_token_id=tokenizer.eos_token_id,
      do_sample=False
  )
  response = outputs[0][input_ids.shape[-1]:]
  result = tokenizer.decode(response, skip_special_tokens=True)
  if result.startswith("Yes"):
    return "Positive", result
  elif result.startswith("No"):
    return "Negative", result
  else:
    return "Neutral", result

## Negative

In [25]:
%%time

for assessment in negative:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: No specific findings can be derived from the diagnosis currently named as the basis for the regulation.
Negative: No. 

The assessment states there are no specific findings related to the diagnosis, therefore no recommendation can be made. 

-----
Assessment: According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.
Positive: Yes. 

The assessment indicates the insured has received the requested product, fulfilling their needs within the specified area of application. 

-----
Assessment: A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.
Negative: No. 

The assessment highlights a lack of justification for the recommended orthopedic aid. The recommendatio

## Positive

In [26]:
%%time

for assessment in positive:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.
Positive: Yes. 

The assessment explicitly states that the need for compensation to ensure basic needs is "conceivable" due to the diagnosis. This indicates a positive recommendation for compensation. 

-----
Assessment: The socio-medical prerequisites for the prescribed aid supply have been met.
Positive: Yes. 

The statement "The socio-medical prerequisites for the prescribed aid supply have been met" indicates that the necessary conditions for providing the prescribed aid have been fulfilled. This implies a positive recommendation for the provision of the aid. 

-----
Assessment: Everyday relevant usage benefits have been determined.
Positive: Yes. 

This statement indicates that the assessment has identified and evaluated the benefits associated with everyday usage, which is a key component of statutory health insurance. 

-----
Assessment: Socio-medical indication for the 

In [28]:
!nvidia-smi

Fri Aug  9 06:30:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0              31W /  70W |   5783MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--